# Transfer Learning Using Pretrained ConvNets

You will follow the general machine learning workflow.

1. Examine and understand the data
2. Build an input pipeline
3. Compose our model
  * Part-1: Load in our pretrained base model (and pretrained weights)
  * Part-2: Stack our classification layers on top
4. Train our model
5. Evaluate model


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os

import numpy as np

import matplotlib.pyplot as plt

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

keras = tf.keras

## Data preprocessing

### Data download

In [0]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

In [0]:
SPLIT_WEIGHTS = (8, 1, 1)
splits = tfds.Split.TRAIN.subsplit(weighted=SPLIT_WEIGHTS)
(raw_train, raw_validation, raw_test), metadata = tfds.load(name="tf_flowers", 
                                                            with_info=True,
                                                            split=list(splits),                                         
                                                            as_supervised=True)

Show the first two images and labels from the training set:

In [0]:
get_label_name = metadata.features['label'].int2str

for image, label in raw_train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

### Format the Data



In [0]:
IMG_SIZE = 160

def format_example(image, label):
  image = tf.cast(image, tf.float32)
  image = (image/255)
  image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
  return image, label

Apply this function to each item in the dataset using the map method:

In [0]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

Now shuffle and batch the data.

In [0]:
BATCH_SIZE = 32
SHUFFLE_BUFFER_SIZE = 1000

In [0]:
train_batches = train.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
validation_batches = validation.batch(BATCH_SIZE)
test_batches = test.batch(BATCH_SIZE)

In [0]:
for image_batch,label_batch in train_batches.take(1):
  pass

image_batch.shape



### Q5 What is the shape of one batch of data?

# 1. Create the base model from the pre-trained convnets
You will create the base model from the **Inception V3** model developed at Google. This is pre-trained on the ImageNet dataset, a large dataset of 1.4M images and 1000 classes of web images. This model has already learned the basic features that are common in 1000 objects that we see daily. Hence, it has a strong feature extraction capability. We download a network that doesn’t include the classification layers at the top, by specifying include_top=False argument, because we only want to use the feature extraction portion of these pre-trained convnets (convolutional base)

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

# Create the base model from the pre-trained model Inception V3
base_model = tf.keras.applications.InceptionV3(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

### Q6 What is the shape of a new block of features converted by the feature extractor?

In [0]:
feature_batch = base_model(image_batch)

feature_batch.shape

## Feature extraction


### Freeze the convolutional base

In [0]:
# Write the code here to freeze convolutional base

base_model.trainable=False

### Add a classification head

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [0]:
prediction_layer = keras.layers.Dense(metadata.features['label'].num_classes, activation='softmax')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

Now stack the feature extractor, and these two layers using a `tf.keras.Sequential` model:

In [0]:
# Write the code here to stack the all these layers

# model = ...

model=tf.keras.Sequential([
    base_model,
    global_average_layer,
    prediction_layer
])

### Q7 What is the total number trainable TF variables in the model?

In [0]:
len(model.trainable_variables)

### Compile the model


In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Q8 What is the total numebr of Trainable parameters in this model?

### Train the model

In [0]:
model.summary()

In [0]:
history = model.fit(train_batches,
                    epochs=10,
                    validation_data=validation_batches)

### Learning curves


In [0]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

# 2. Fine tuning


### Un-freeze the top layers of the model


In [0]:
# Write code here for un-freezing the top layers
base_model.trainable=True

In [0]:
# Write the code for freezing layers before layer 250 in the model
print(len(base_model.layers))
fine_tune_at=250

for layer in base_model.layers[:fine_tune_at]:
  layer.trainable=False
  

### Q9 What is the total number trainable TF variables in the model?

### Compile the model


In [0]:
len(model.trainable_variables)

In [0]:
model.compile(optimizer = 'adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

### Q10 What is the total numebr of Trainable parameters in this model now?

In [0]:
model.summary()

### Continue Train the model

In [0]:
initial_epochs=10
total_epochs = initial_epochs + 10

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch = initial_epochs,
                         validation_data=validation_batches)

### Q11 Let the training accuracy in part-1 be x and training accuracy in part-2 be y (both after training is done). The value of | y-x | lies between?

In [0]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

In [0]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()